In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [5]:
# データセットのパス
DATA_PATH = './data/microplastics/datasets_full_v2'
IMAGE_DIR = os.path.join(DATA_PATH, 'image')
LABEL_DIR = os.path.join(DATA_PATH, 'label')

IMG_HEIGHT = 512
IMG_WIDTH = 512
BATCH_SIZE = 32

In [6]:
# データ拡張の設定
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest',
                    validation_split=0.1)

# ImageDataGeneratorの作成
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

In [7]:
# データジェネレータの作成
seed = 1  # シード値を固定して画像とマスクの対応を維持
image_generator = image_datagen.flow_from_directory(
    IMAGE_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode=None, # クラス分類ではないためNone
    seed=seed,
    color_mode='grayscale', # グレースケール画像として読み込む
    subset='training')

mask_generator = mask_datagen.flow_from_directory(
    LABEL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed=seed,
    color_mode='grayscale',
    subset='training')

validation_image_generator = image_datagen.flow_from_directory(
    IMAGE_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed=seed,
    color_mode='grayscale',
    subset='validation')

validation_mask_generator = mask_datagen.flow_from_directory(
    LABEL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode=None,
    seed=seed,
    color_mode='grayscale',
    subset='validation')


# ジェネレータを結合
train_generator = zip(image_generator, mask_generator)
val_generator = zip(validation_image_generator, validation_mask_generator)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [8]:
# U-Netモデルの定義
def unet(input_size=(IMG_HEIGHT, IMG_WIDTH, 1)):
    inputs = keras.Input(input_size)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Bottleneck
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)

    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    concat1 = layers.concatenate([up1, conv2], axis=-1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(concat1)
    conv4 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv4)

    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    concat2 = layers.concatenate([up2, conv1], axis=-1)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(concat2)
    conv5 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv5)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv5) # 出力は1チャンネル（マスク）
    return keras.Model(inputs=inputs, outputs=outputs)

In [9]:
# モデルのコンパイル
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Unpack the generators
train_images, train_masks = zip(*train_generator)
val_images, val_masks = zip(*val_generator)

# Use the unpacked data in model.fit
model.fit(train_images, train_masks, epochs=10, 
          validation_data=(val_images, val_masks), 
          steps_per_epoch=len(image_generator), 
          validation_steps=len(validation_image_generator))

In [ ]:
# モデルの保存
model.save('unet_microplastic.h5')